In [ ]:
import os       #文件操作的库
import zipfile   #文件解压的库
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop    #网络训练，优化算法
from tensorflow.keras.preprocessing.image import ImageDataGenerator    #图的预处理
from shutil import copyfile  #对文件进行复制

In [1]:
# If the URL doesn't work, visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL to the dataset

# Note: This is a very large dataset and will take time to download

!wget --no-check-certificate \      #wget进行数据下载
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"     #存在云服务器了，记得改路径

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')    #只读
zip_ref.extractall('/tmp')    #解压到此
zip_ref.close()    #解压后关闭


'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


NameError: name 'zipfile' is not defined

In [ ]:
print(len(os.listdir('/tmp/PetImages/Cat/')))   #有len看下有多少数据
print(len(os.listdir('/tmp/PetImages/Dog/')))

# Expected Output:
# 12501
# 12501

In [ ]:
try:                          #对训练和测试的狗毛猫数据分别放到一个独立的文件夹里面
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

In [ ]:
import os
import shutil

#把解压后的数据分为测试机和训练集，SPLIT_SIZE是划分比例
#SOURCE 就是解压后存储训练和测试数据文件的文件夹
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):   
        file = SOURCE + filename     #这个有意思，文件名拼接打开对应文件
        if os.path.getsize(file) > 0:   #是非空图片的话存一下
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SOURCE)    #SOURCE 是0.9的话就说明训练数据占有90%
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))   #把图片顺序打乱
    training_set = shuffled_set[0:training_length]   #切片抽取训练样本集
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:    #读取训练样本集
        this_file = SOURCE + filename    #SOURCE 是存放狗猫数据原路径，加上训练集的名字
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:  #读取测试样本集
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)   #用这个命令读进来，读到训练样本集中，读进去，放到destination


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"            #猫的数据
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"   #训练样本的目录
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"     #测试样本的目录
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"            #狗的数据
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

def create_dir(file_dir):      #产生路径（文件夹）的命令
    if os.path.exists(file_dir):    #如果路径存在
        print('true')
        #os.rmdir(file_dir)
        shutil.rmtree(file_dir)#删除
        os.makedirs(file_dir)  #再建立
    else:
        os.makedirs(file_dir)

#前面已经生成，再做确认而已
#生成四个子目录，存在的话会显示 True，删除再建立一个空的，没有的话就再建立
create_dir(TRAINING_CATS_DIR)
create_dir(TESTING_CATS_DIR)
create_dir(TRAINING_DOGS_DIR)
create_dir(TESTING_CATS_DIR)
    
#做调用切分的，切分比例为 9:1
split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

In [ ]:
#os.listdir 列出本路径下有多少文件（图片）
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

# Expected output:
# 11250
# 11250
# 1250
# 1250

In [1]:
#搭建神经网络
model = tf.keras.models.Sequential([
    #Sequential 表示每一行为一层，每层之间是顺序连接的
    #16 32 64 512 1 都表示通道，随着隐层的加深，通道的数量也是逐步增加的
    #3*3 是卷积核的大小
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),  #卷积
    tf.keras.layers.MaxPooling2D(2, 2),                                                #最大池化
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),         #把最后的最大池化层拉直了？
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),   #接一个全连接的影层
    tf.keras.layers.Dense(1, activation='sigmoid')   #输出层
    #因为是猫狗二分类，所以输出层神经元可以使一个也可以是两个
    #如果输出接近 1，认为是狗；接近 0， 认为是猫
    #输出层用的 sigmoid 激活函数，其他用 relu 做激活函数
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
#训练步长初始为 0.001，用 RMSprop 来修订步长；这个网络的损失函数，用了一个二元的交叉熵；网络性能的度量，用的是准确度

NameError: name 'tf' is not defined

In [ ]:
#这里是要做图像预处理
TRAINING_DIR = "/tmp/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)  #Imag...把训练的样本集做规划为 0~1 之间，减少计算量
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,   #100 个样本作为一批，共 n/100 批
                                                                      #一个 epochs 就是总共的训练样本除以100
                                                                      #batch_size 就是我们一个 epochs 要循环的轮次
                                                    class_mode='binary',   #类别为 binary 这是二分类
                                                    target_size=(150, 150))  #把图片做成 150*150 的

VALIDATION_DIR = "/tmp/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=100,  
                                                              class_mode='binary',
                                                              target_size=(150, 150))

# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

In [ ]:
# Note that this may take some time.
history = model.fit_generator(train_generator,
                              epochs=2,     #epochs=2 指要训练两轮，可以自己改数值
                              verbose=1,    #是否要记录，1 表示要
                              validation_data=validation_generator)
#一轮表示所有样本都要参加梯度下降，每批只能对 100 个样本进行训练，所以要进行 225 批训练（题目而定）

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']      #计算训练样本准确度
val_acc=history.history['val_acc']  #计算测试样本准确度
loss=history.history['loss']     #计算训练样本损失函数
val_loss=history.history['val_loss']  #计算测试样本损失函数

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()


# Desired output. Charts with training and validation metrics. No crash :)

In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))   #加载新的图片
  x = image.img_to_array(img)   #由列表变成向量的形式，X 是一个多维的数组
  x = np.expand_dims(x, axis=0)   #拉直成相应的向量

  images = np.vstack([x])    用 vatack 按照水平方向把各个通道的数据连接起来构成一个长向量
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5: 
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")